# A full training

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [18]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# checkpoint — задает модель, для которой будет загружен токенизатор (BERT-base без учета регистра).
# AutoTokenizer.from_pretrained(checkpoint) — автоматически загружает токенизатор для указанной модели.
# Преобразует текст в токены (например, "Hello world!" → ["hello", "world", "!"]).
# Преобразует токены в числовые идентификаторы (ID), понятные модели.
# Добавляет специальные токены ([CLS], [SEP]) для модели BERT.

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Пример работы токенизатора:
```
example = {"sentence1": "This is a sentence.", "sentence2": "This is another sentence."}
tokenize_function(example)
```
Выход:
```
{
    'input_ids': [101, 2023, 2003, 1037, 6251, 1012, 102, 2023, 2003, 2178, 6251, 1012, 102],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],  # Маркировка двух предложений
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}
```
- input_ids — числовые идентификаторы токенов.
- token_type_ids — указывает, какие токены принадлежат какому предложению (0 — первое, 1 — второе).
- attention_mask — маска для обозначения важных токенов (1) и паддинга (0).

- batched=True означает, что функция получает сразу пакет примеров (batch), а не по одному, что ускоряет обработку.

- DataCollatorWithPadding автоматически добавляет паддинг к разной длины входным данным в батче.
Это полезно, так как предложения в датасете могут иметь разную длину, а модели требуют одинаковых размеров входов.

In [19]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [20]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

мпортируется класс `DataLoader` из модуля `torch.utils.data`, который помогает разбивать датасет на батчи и перемешивать данные для обучения.  
1. Создание загрузчика для обучающего набора
```
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
```
- Используется обучающая часть токенизированного датасета tokenized_datasets["train"].  
- shuffle=True: данные перемешиваются перед формированием батчей, что помогает модели лучше обучаться.  
- batch_size=8: размер каждого батча равен 8 примерам.  
- collate_fn=data_collator: функция, которая автоматически дополняет (padding) данные до одинаковой длины внутри батча.

2. Создание загрузчика для валидационного набора  
```
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)
```
Здесь аналогично создается загрузчик для валидационного датасета tokenized_datasets["validation"].
Данные не перемешиваются (по умолчанию shuffle=False), поскольку при валидации важна последовательность примеров.
Размер батча и функция collate_fn аналогичны обучающему набору.

In [21]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 69]),
 'token_type_ids': torch.Size([8, 69]),
 'attention_mask': torch.Size([8, 69])}

Этот цикл берет первый батч из train_dataloader и сразу выходит (break).  
Вывод формы тензоров в батче
```{k: v.shape for k, v in batch.items()}```  
- batch — это словарь, содержащий тензоры с данными (например, input_ids, attention_mask, token_type_ids, labels).
Словарь раскладывается в формате `{ключ: форма_тензора}`, чтобы показать размеры всех тензоров.

мы используем batch_size=8, тогда вывод:
```
{
    'input_ids': torch.Size([8, 128]),
    'attention_mask': torch.Size([8, 128]),
    'token_type_ids': torch.Size([8, 128]),
    'labels': torch.Size([8])
}
```
- [8, 128] означает, что в батче 8 примеров, каждый имеет последовательность длиной 128 токенов.
- labels (метки классов) имеет размер [8], так как для каждого примера есть одно число (0 или 1 в задаче MRPC).

Оператор `break` используется здесь, чтобы остановить цикл после первой итерации, то есть получить только первый батч из train_dataloader и не обрабатывать весь датасет.

In [22]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6783, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


мы передаем батч данных в модель.
- **batch — это распаковка словаря (batch содержит input_ids, attention_mask, token_type_ids и labels).  
- model(**batch) эквивалентно model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"]).  
Выход модели (outputs) — это объект SequenceClassifierOutput, содержащий:
- loss (если передан labels).
- logits (сырые предсказания модели).  

- `outputs.loss` - loss (функция потерь) будет в outputs только если переданы labels.
В задаче бинарной классификации (MRPC) используется CrossEntropyLoss.
Значение loss — это одно число (torch.Tensor), например:
`tensor(0.6931, grad_fn=<NllLossBackward>)`  
Используется для обратного распространения ошибки (loss.backward()).

- `outputs.logits.shape` logits — это сырые выходы модели перед softmax.
Форма logits.shape: `torch.Size([batch_size, num_labels])`
Например, если batch_size=8 и num_labels=2: `torch.Size([8, 2])`
Это по 2 логита (одно число на каждый класс) для каждого из 8 примеров в батче.

Чтобы `получить вероятности классов`, нужно применить softmax:
```
import torch
probabilities = torch.softmax(outputs.logits, dim=1)
```

Мы почти готовы написать наш цикл обучения! Нам не хватает только двух вещей: оптимизатора и планировщика скорости обучения. Поскольку мы пытаемся вручную повторить то, что делал Trainer, мы будем использовать те же значения по умолчанию. Оптимизатор, используемый Trainer, — AdamW, который является тем же самым, что и Adam, но с изюминкой для регуляризации распада веса (см. «Decoupled Weight Decay Regularization» Ильи Лощилова и Фрэнка Хаттера):

In [24]:
# from transformers import AdamW
from torch.optim import AdamW  # Используем PyTorch версию
optimizer = AdamW(model.parameters(), lr=5e-5)

Наконец, планировщик скорости обучения, используемый по умолчанию, представляет собой просто линейный спад от максимального значения (5e-5) до 0. Чтобы правильно его определить, нам нужно знать количество шагов обучения, которые мы сделаем, то есть количество эпох, которые мы хотим запустить, умноженное на количество партий обучения (что является длиной нашего загрузчика данных обучения). Тренер использует три эпохи по умолчанию, поэтому мы будем следовать этому:

In [25]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


Функция `get_scheduler` из библиотеки transformers помогает создать планировщик для изменения скорости обучения во время обучения модели.

1. Определение количества эпох и шагов обучения
- num_epochs = 3 — обучение модели будет проходить в 3 эпохи.  
- num_training_steps = num_epochs * len(train_dataloader) — общее количество шагов обучения вычисляется как число эпох, умноженное на количество батчей в train_dataloader (то есть, сколько раз оптимизатор обновит веса модели).  
3. Создание планировщика скорости обучения
- optimizer=optimizer,   Оптимизатор, которому применяется scheduler  
- num_warmup_steps=0,    # Количество шагов разогрева (warm-up)
- num_training_steps=num_training_steps,  количество шагов, за которые learning rate снизится до 0

Итог  
Определяет общее количество шагов обучения.
Создает линейный планировщик изменения learning rate.
Позволяет контролировать скорость обучения, уменьшая ее по мере обучения.

In [26]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
# Определяет, доступен ли GPU (torch.cuda.is_available()).
# Переносит модель на GPU, что значительно ускоряет вычисления.

print(device)
print(torch.cuda.get_device_name(0))  # Должно показать "T4" или другую видеокарту

cuda
Tesla T4


Теперь мы готовы к обучению! Чтобы иметь некоторое представление о том, когда обучение будет завершено, мы добавляем полосу прогресса над количеством шагов обучения, используя библиотеку `tqdm`:

In [27]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

Этот код выполняет обучение модели с использованием __PyTorch__ и библиотеки __tqdm__ для отображения прогресса обучения. Давайте разберем его шаг за шагом.  

1. Импорт и создание прогресс-бара __from tqdm.auto import tqdm__
`progress_bar = tqdm(range(num_training_steps))`  
Импортируется tqdm.auto, который автоматически выбирает правильный прогресс-бар для Jupyter Notebook или консоли.
- progress_bar = tqdm(range(num_training_steps)) создает индикатор выполнения (progress_bar), который показывает, сколько шагов обучения выполнено.

2. Переключение модели в режим обучения  
`model.train()` - Переключает модель в режим обучения (training mode), что включает поведение, специфичное для обучения, например, включение dropout.  


3. Цикл обучения  
- `for epoch in range(num_epochs)`: запускает цикл по количеству эпох.
- `for batch in train_dataloader`: проходит по батчам (пакетам данных) из train_dataloader.  
4. Перемещение данных на GPU (если используется)  
`batch = {k: v.to(device) for k, v in batch.items()}`
5. Прямой проход (forward pass)  
- `outputs = model(**batch)` - Пропускает батч через модель.
- `loss = outputs.loss` - Извлекает значение функции потерь (loss).  
6. Обратное распространение ошибки (backpropagation)
- `loss.backward()` - Вычисляет градиенты ошибки относительно параметров модели.  
7. Шаг оптимизации  
- `optimizer.step()` - Обновляет параметры модели с учетом вычисленных градиентов.  
8. Обновление learning rate
- `lr_scheduler.step()` - Планировщик lr_scheduler обновляет learning rate (скорость обучения) согласно выбранной стратегии.  
9. Обнуление градиентов
- `optimizer.zero_grad()` - Очищает градиенты, чтобы они не накапливались между батчами.  
10. Обновление прогресс-бара  
- `progress_bar.update(1)` - Обновляет прогресс-бар на 1 шаг, отображая выполнение процесса обучения.

Итог  
Этот код:  
✅ Запускает обучение модели на num_epochs эпох.  
✅ Проходит по данным из train_dataloader.  
✅ Выполняет прямой проход, вычисляет функцию потерь, делает обратное распространение ошибки.  
✅ Обновляет параметры модели с помощью оптимизатора.  
✅ Корректирует скорость обучения через планировщик.  
✅ Отображает прогресс обучения с tqdm.  

## Цикл оценки

Как и раньше, мы будем использовать метрику, предоставленную библиотекой 🤗 Evaluate. Мы уже видели метод metric.compute(), но метрики могут фактически накапливать пакеты для нас, когда мы проходим цикл прогнозирования с помощью метода add_batch(). После того, как мы накопим все пакеты, мы можем получить окончательный результат с помощью metric.compute(). Вот как реализовать все это в цикле оценки:

In [28]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8774509803921569, 'f1': 0.9131944444444444}

- `metric = evaluate.load("glue", "mrpc")` - Загружает метрику, специфичную для MRPC (Microsoft Research Paraphrase Corpus). MRPC — это задача бинарной классификации, определяющая, являются ли два предложения перефразами друг друга.  
- `model.eval()` - Переводит модель в режим оценки (evaluation mode), отключая dropout и другие механизмы, используемые только при обучении.  
- ```
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}4. Цикл по данным из eval_dataloader
```
Итерация по eval_dataloader, который содержит батчи данных для тестирования.
Перемещает данные (batch) на GPU или CPU в зависимости от device.  
- ```
with torch.no_grad():
    outputs = model(**batch)
```    
Отключение вычисления градиентов - Используется torch.no_grad(), чтобы сэкономить память и ускорить вычисления, так как градиенты в процессе оценки не нужны. Прогоняет batch через модель, получая outputs.
__Предсказания модели__  
- `logits = outputs.logits`
- `predictions = torch.argmax(logits, dim=-1)`
- `logits = outputs.logits` — извлекает логиты (сырые выходные значения перед софтмаксом).
- `torch.argmax(logits, dim=-1)` — выбирает самый вероятный класс из предсказаний.
__Добавление предсказаний в метрику__
- `metric.add_batch(predictions=predictions, references=batch["labels"])` - Добавляет предсказания и реальные метки (labels) для дальнейшего вычисления метрик.
_Вычисление метрики__
- `metric.compute()` - Вычисляет итоговые метрики (например, Accuracy, F1, Precision, Recall) на основе собранных предсказаний и реальных значений.

Что делает код в целом? 🚀  
Этот код:  
✅ Прогоняет тестовые данные через модель без градиентов.  
✅ Собирает предсказания и истинные метки.  
✅ Вычисляет метрики GLUE MRPC, такие как F1-score и Accuracy.

<font color= 'lightgreen'>Все сразу ...

In [ ]:
from transformers import AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

<font color= 'lightgreen'>Цикл обучения, который мы определили ранее, отлично работает на одном CPU или GPU. Но с помощью библиотеки 🤗 Accelerate, всего с несколькими корректировками, мы можем включить распределенное обучение на нескольких GPU или TPU. Начиная с создания загрузчиков данных обучения и проверки, вот как выглядит наш цикл обучения вручную:

In [29]:
from accelerate import Accelerator
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1377 [00:00<?, ?it/s]

Если вы хотите скопировать и вставить его, чтобы поиграться, вот как выглядит полный цикл обучения с 🤗 Accelerate:

- Первая строка для добавления — это строка импорта.  
- Вторая строка создает экземпляр объекта Accelerator, который будет просматривать среду и инициализировать правильную распределенную настройку. 🤗 Accelerate обрабатывает размещение устройства за вас, поэтому вы можете удалить строки, которые помещают модель на устройство (или, если хотите, изменить их на использование accelerator.device вместо device).

- Затем основная часть работы выполняется в строке, которая отправляет загрузчики данных, модель и оптимизатор в accelerator.prepare(). Это обернет эти объекты в правильный контейнер, чтобы убедиться, что ваше распределенное обучение работает так, как задумано.  
- Осталось внести изменения, удалив строку, которая помещает пакет на устройство (опять же, если вы хотите сохранить это, вы можете просто изменить его на использование accelerator.device) и заменить loss.backward() на accelerator.backward(loss).

In [30]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8480392156862745, 'f1': 0.8934707903780069}

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)